# 102Flowers Image Classifier

This is the main notebook for the project. See the associated report (WIP) for more information.

WORK IN PROGRESS

### Imports

In [12]:
import torch
from torch import nn
from torch import optim
from torch import argmax
from torch.utils.data import DataLoader
from torch.utils.data import ConcatDataset
import torchvision.transforms as trnfrm
import torch.nn.functional as F
from torchvision import datasets, transforms, models

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import string
import random
import statistics
from statistics import mean
from statistics import stdev

### Hyperparameters

In [13]:
# TODO: Set hyperparameters.
training_batch_size = 20 #1020
validation_batch_size = 20 #1020
test_batch_size = 20 #6149
epochs = 690
learning_rate = 0.001
lambda1 = 0.0001
weightDecay = 0.001
#momentum = 0.9
crop_size = 128

### Device

In [14]:
# Default to CPU
device = torch.device("cpu")

# Switch to GPU if available
if torch.cuda.is_available():
	print(f"Found {torch.cuda.device_count()} GPUs. Using cuda:0.")
	device = torch.device("cuda:0")
else:
	print("No GPUs found, using CPU.")

Found 1 GPUs. Using cuda:0.


### Load Dataset

In [15]:
global mean
original_training_data = datasets.Flowers102(
    root = "data",
    split = "train",
    transform=transforms.Compose([
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
        #transforms.Normalize((0.4703, 0.3986, 0.3176), (0.2957, 0.2464, 0.2804))
    ]),
    download=True
)

h_flipped_training_data = datasets.Flowers102(
    root = "data",
    split = "train",
    transform=transforms.Compose([
        transforms.RandomHorizontalFlip(0.99),
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
        #transforms.Normalize((0.4703, 0.3986, 0.3176), (0.2957, 0.2464, 0.2804))
    ]),
    download=True
)

v_flipped_training_data = datasets.Flowers102(
    root = "data",
    split = "train",
    transform=transforms.Compose([
        transforms.RandomVerticalFlip(0.99),
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
        #transforms.Normalize((0.4703, 0.3986, 0.3176), (0.2957, 0.2464, 0.2804))
    ]),
    download=True
)

pos90_rotate_training_data = datasets.Flowers102(
    root = "data",
    split = "train",
    transform=transforms.Compose([
        transforms.RandomRotation([89,91]),
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
        #transforms.Normalize((0.4703, 0.3986, 0.3176), (0.2957, 0.2464, 0.2804))
    ]),
    download=True
)

minus90_rotate_training_data = datasets.Flowers102(
    root = "data",
    split = "train",
    transform=transforms.Compose([
        transforms.RandomRotation([-91,-89]),
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
        #transforms.Normalize((0.4703, 0.3986, 0.3176), (0.2957, 0.2464, 0.2804))
    ]),
    download=True
)

skewed1_training_data = datasets.Flowers102(
    root = "data",
    split = "train",
    transform=transforms.Compose([
        transforms.RandomPerspective(p = 0.99),
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ]),
    download=True
)

skewed2_training_data = datasets.Flowers102(
    root = "data",
    split = "train",
    transform=transforms.Compose([
        transforms.RandomPerspective(0.3,p = 0.99),
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ]),
    download=True
)

augmented_training_data = ConcatDataset([original_training_data,h_flipped_training_data,v_flipped_training_data,pos90_rotate_training_data,minus90_rotate_training_data,skewed1_training_data,skewed2_training_data])

validation_data = datasets.Flowers102(
    root = "data",
    split = "val",
    transform=transforms.Compose([
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ]),
    download=True
)

testing_data = datasets.Flowers102(
    root = "data",
    split = "test",
    transform=transforms.Compose([
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ]),
    download=True
)

### DataLoaders

In [16]:
train_dataloader = DataLoader(augmented_training_data, batch_size=training_batch_size, shuffle=True)
#train_dataloader = DataLoader(original_training_data, batch_size=training_batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=validation_batch_size, shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size=test_batch_size, shuffle=True)

## Model

In [17]:
classifications = F.one_hot(torch.tensor([e for e in range(0,102)], device="cuda:0"), num_classes=102)
classifications, classifications.size()

(tensor([[1, 0, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0],
         [0, 0, 1,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 1, 0, 0],
         [0, 0, 0,  ..., 0, 1, 0],
         [0, 0, 0,  ..., 0, 0, 1]], device='cuda:0'),
 torch.Size([102, 102]))

In [18]:
class F102Classifier(nn.Module):
    
    """Proper
    def __init__(self):
        super(F102Classifier, self).__init__()
        
        self.pool = nn.MaxPool2d(3, 2)
        self.conv1 = nn.Conv2d(3, 16, 3, bias = False, groups = 1) #3 inputs 6 hiddens
        self.batchnorm1 = nn.BatchNorm2d(16) #Normalizes above 
        self.conv2 = nn.Conv2d(16, 32, 3, bias = False, groups = 1) # 12 hiddens
        self.batchnorm2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, 3, bias = False, groups = 1) # 12 hiddens
        self.batchnorm3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 128, 3, bias = False, groups = 1) # 12 hiddens
        self.batchnorm4 = nn.BatchNorm2d(128)
        self.conv5 = nn.Conv2d(128, 256, 3, bias = False, groups = 1) # 12 hiddens
        self.batchnorm5 = nn.BatchNorm2d(256)
        self.fc1 = nn.Linear(6400, 6400)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(6400, 102) #102 output neurons
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.batchnorm1(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.batchnorm2(x)
        x = self.pool(F.relu(self.conv3(x)))
        x = self.batchnorm3(x)
        x = self.pool(F.relu(self.conv4(x)))
        x = self.batchnorm4(x)
        x = self.pool(F.relu(self.conv5(x)))
        x = self.batchnorm5(x)
        x = x.view(training_batch_size, -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
    #"""
    #"""
    def __init__(self):
        super(F102Classifier, self).__init__()
        
        self.pool = nn.MaxPool2d(3, 2)
        self.conv1 = nn.Conv2d(3, 32, 3, bias = False, groups = 1) #3 inputs 6 hiddens
        self.batchnorm1 = nn.BatchNorm2d(32) #Normalizes above 
        self.conv2 = nn.Conv2d(32, 64, 3, bias = False, groups = 1) # 12 hiddens
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 64, 3, bias = False, groups = 1) # 12 hiddens
        self.batchnorm3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 96, 3, bias = False, groups = 1) # 12 hiddens
        self.batchnorm4 = nn.BatchNorm2d(96)
        self.conv5 = nn.Conv2d(96, 96, 3, bias = False, groups = 1) # 12 hiddens
        self.batchnorm5 = nn.BatchNorm2d(96)
        self.fc1 = nn.Linear(2400, 2400)
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(2400, 102) #102 output neurons
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.batchnorm1(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.batchnorm2(x)
        x = self.pool(F.relu(self.conv3(x)))
        x = self.batchnorm3(x)
        x = self.pool(F.relu(self.conv4(x)))
        x = self.batchnorm4(x)
        x = self.pool(F.relu(self.conv5(x)))
        x = self.batchnorm5(x)
        x = x.view(training_batch_size, -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
    
    def lambda1_loss(self, n):
      return torch.abs(n).sum()
    #"""
net = F102Classifier()
if device == torch.device("cuda:0"):
    net.cuda()

### Loss Function & Optimiser

In [19]:
loss_function = nn.CrossEntropyLoss()
optimiser = optim.Adam(net.parameters(), lr=learning_rate, weight_decay=weightDecay,)

## Train

In [20]:
def train(dataloader, model, loss_fn, optimizer):
    avgLoss = 0
    for batch, (i,j) in enumerate(dataloader):

        features, labels = i.to(device), j.to(device)

        l1_parameters = []
        for parameter in model.parameters():
          l1_parameters.append(parameter.view(-1))
        l1_regularization = lambda1 * model.lambda1_loss(torch.cat(l1_parameters))

        #all_linear1_params = torch.cat([x.view(-1) for x in net.fc1.parameters()])
        #l1_regularization = lambda1 * torch.norm(all_linear1_params, 1)

        # Compute the loss based off the predictions vs labels
        predictions = model(features)
        loss = loss_fn(predictions, labels) + l1_regularization
        avgLoss += loss

        #Compute back propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (batch+1) % len(dataloader) == 0: #after final batch of each epoch
            avgLoss = avgLoss / training_batch_size
            #print("lamda1: " + str(l1_regularization))
            print(f'Average Training Loss: {loss.item()}\n')

### Testing

In [21]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)# - 1
    model.cuda()
    model.eval()
    test_loss, correct = 0, 0
    #n = 0
    for batch, (i,j) in enumerate(dataloader):
        #n += 1
        #if n < num_batches:

        features, labels = i.to(device), j.to(device)
        model.cuda()
        pred = model(features)
        test_loss += loss_fn(pred, labels).item()
        #for i in range(19):
            #if labels[i] !=argmax(pred[i]):
                #print("this label " + str(labels[i]) + " missclassed as" + str(argmax(pred[i])))
        correct += (pred.argmax(1) == labels).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size

    print(f"Accuracy: {(100*correct):>0.1f}%, Avg loss : {test_loss:>8f} \n")
    return correct

### Running Training and Testing And saving

In [22]:
#"""
best = 0
upperLetters = string.ascii_uppercase
lowerLetters = string.ascii_lowercase
name = random.choice(upperLetters) + random.choice(lowerLetters) + random.choice(lowerLetters)
print("Random Name of Model is " + name)

for t in range(epochs):
    print(f'Epoch {t+1}-------------')
    train(train_dataloader, net, loss_function, optimiser)
    #print("Training Test")
    #test(train_dataloader, net, loss_function)
    print("Validation Test")
    current = test(validation_dataloader, net, loss_function)
    if current > best:
        print("new best, saving model for epoch " + str(t+1))
        best = current
        save_path = "./models/classifier" + name+  "Epoch" + str(t+1) + "Accuracy" + str(int(100*best))+ ".pth"
        torch.save(net.state_dict(), save_path)
print('Finished Training, Testing and Saving')
#"""
"""
Pmodel = F102Classifier()
Pmodel.load_state_dict(torch.load("./models/classifierTmkEpoch186Accuracy54.pth"))
test(validation_dataloader, Pmodel, loss_function)
#test(test_dataloader, Pmodel, loss_function) 
#"""

Random Name of Model is Geu
Epoch 1-------------
Average Training Loss: 6.050067901611328

Validation Test
Accuracy: 26.9%, Avg loss : 3.522724 

new best, saving model for epoch 1
Epoch 2-------------
Average Training Loss: 4.125741004943848

Validation Test
Accuracy: 37.7%, Avg loss : 2.939497 

new best, saving model for epoch 2
Epoch 3-------------
Average Training Loss: 3.8035507202148438

Validation Test
Accuracy: 44.0%, Avg loss : 2.587480 

new best, saving model for epoch 3
Epoch 4-------------
Average Training Loss: 2.848842144012451

Validation Test
Accuracy: 43.3%, Avg loss : 2.654286 

Epoch 5-------------
Average Training Loss: 2.496342658996582

Validation Test
Accuracy: 47.9%, Avg loss : 2.656653 

new best, saving model for epoch 5
Epoch 6-------------
Average Training Loss: 2.286550283432007

Validation Test
Accuracy: 45.7%, Avg loss : 2.980349 

Epoch 7-------------
Average Training Loss: 2.143317937850952

Validation Test
Accuracy: 45.4%, Avg loss : 2.807081 

Epoch

KeyboardInterrupt: 

### Record Keeping

""" 30,0.001 = 15.2%
 100, 0.001 = 14.3%
 50, 0.001 = 16.4%
 30,0.01 = 1.0%  Herma-OF"""

"""For Theo	

        self.pool = nn.AvgPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 6, 3) 
		self.conv2 = nn.Conv2d(6, 12, 3)
		self.conv3 = nn.Conv2d(12, 24, 3)
		self.conv4 = nn.Conv2d(24, 48, 3)
		self.conv5 = nn.Conv2d(48, 96, 3)
		self.fc1 = nn.Linear(384, 1024)
		self.fc2 = nn.Linear(1024, 512)
		self.fc3 = nn.Linear(512, 102)	
30, 0.01 = 1.0%
30, 0.001 = 14.0%
100, 0.001 = 13.2%
30, 0.0001 = 7.2%

		self.pool = nn.AvgPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 12, 3)
		self.conv2 = nn.Conv2d(12, 48, 3)
		self.conv3 = nn.Conv2d(48, 96, 3)
		self.fc1 = nn.Linear(18816, 1024)
		self.fc2 = nn.Linear(1024, 512)
		self.fc3 = nn.Linear(512, 102)		
100, 0.001 = 9.5%
30, 0.001 = 17.8%
50, 0.001 = 1.0%

		self.pool = nn.AvgPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 12, 3)
		self.fc1 = nn.Linear(47628, 102)
30, 0.001 = 19.9%

		self.pool = nn.AvgPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 6, 3)
		self.fc1 = nn.Linear(23814, 102)
30, 0.001 = 17.9%
15, 0.001 = 16.0%
10, 0.001 = 19.0%
5, 0.001 = 14.4%

		self.pool = nn.MaxPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 6, 3)
		self.fc1 = nn.Linear(23814, 102)
30, 0.001 = 19.2%
10, 0.001 = 16.8%
50, 0.001 = 17.0

		self.pool = nn.MaxPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 6, 3)
		self.conv2 = nn.Conv2d(6, 12, 3)
		self.conv3 = nn.Conv2d(12, 24, 3)
		self.conv4 = nn.Conv2d(24, 48, 3)
		self.conv5 = nn.Conv2d(48, 96, 3)
		self.fc1 = nn.Linear(384, 102)
30, 0.001 = 18.6%

MORE
self.pool = nn.MaxPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 6, 3)
		self.conv2 = nn.Conv2d(6, 12, 3)
		self.conv3 = nn.Conv2d(12, 24, 3)
		self.fc1 = nn.Linear(21600, 102)
	def forward(self, x):
		x = (F.relu(self.conv1(x)))
		x = self.pool(F.relu(self.conv2(x)))
		x = self.pool(F.relu(self.conv3(x)))
		x = x.view(training_batch_size, -1)
		x = self.fc1(x)
        5 epochs = 10.5%
        10 epochs = 17.2%
        30 epochs = 16.1%
        15 epochs = 10.9
        
        Same as above but kernel size 4 and self.fc1 = nn.Linear(18816, 102)
        10 epochs = 9.2%
        15 epochs = 19.2%
        20 epochs = 21.5%
        25 epochs = 14.5%
        30 epochs = 18.4%
        
        Same as above but kernal size 8 and self.fc1 = nn.Linear(15000, 102)
        20 epochs = 15.4 %
        30 epochs = 25.5%
        35 epochs = 16.1%
        
        Same as above but with batch normilization after conv1
        20 epochs = 12.9%
        25 epochs = 25.0%
        30 epochs = 19.6%
        40 epochs = 18.0%
        
        Same as above but with dropout of 0.5 after conv1 and batch normilization
        10 epochs = 7.3%
        15 epochs = 9.1%
        20 epochs = 12.4%
        30 epochs = 10.3%
        40 epochs = 11.6%
        
        Same as above but dropout set to 0.2
        20 epochs = 10.5%
        25 epochs = 16.2%
        30 epochs = 16.7%
        32 epochs = 12.6%
        35 epochs = 19.1%
        40 epochs = 13.5%
        50 epochs = 9.3%
        
        Now have implemented testing per epoch, now will just record the peak epoch.

        Same as above but with conv3 removed and self.fc1 = nn.Linear(8112, 102)
        50 epochs = 28.0%
        
        Same as above but modified conv2 to have 6 outputs and nn.Linear(4056, 102)
        20 epochs = 20.0%
        Reverting to previous conv2 and fcl
        
        Now reverted, applying data augmentation: horizontal flip, vertical flip, 90 degree rotates, -90 degree rotates
        
        Turns out it wasn't properly applying augmentations, I'm using less exact values for the randomness so it basically always does it.
        4 epochs = 32.5%

        Same as above but without the vertical flipping
        5 epochs = 30.9%
        It seems vertical flipping is beneficial

        Same as before but with vertical flipping back and additionally 45 and -45 rotation augments
        4 epochs = 32.3%

        Same as before but removing both 45 degree rotation augments and setting dropout rate at 0.3
        3 epochs = 29%

        Same as before but setting dropout rate to 0.1
        2 epochs = 20.2%

        Same as before but setting dropout rate to 0.25
        6 epochs = 26.7%

        Restting dropout rate back to 0.2 and now testing it on the training data to see if the loss function is the issue
        6 epochs 31.4%
        Loss function is maybe fine

        same as before but removing batch normilization
        9 epochs = 31.7%
        without normilization accuracy increases slower but is more consistently high, will continue without it for now

        Same as before but pool kernal of 4 self.fc1 = nn.Linear(10092, 102)
        20 epochs = 30.8%
        Current model as follows:
        self.pool = nn.MaxPool2d(8, 2)
		self.conv1 = nn.Conv2d(3, 6, 3)
		#self.batchnorm1 = nn.BatchNorm2d(6)
		self.dropout = nn.Dropout2d(0.2)
		self.conv2 = nn.Conv2d(6, 12, 3)
		#self.conv3 = nn.Conv2d(3, 24, 3)
		self.fc1 = nn.Linear(8112, 102)
	def forward(self, x):
		x = self.pool(F.relu(self.conv1(x)))
		#x = self.batchnorm1(x)
		x = self.dropout(x)
		x = self.pool(F.relu(self.conv2(x)))
		x = x.view(training_batch_size, -1)
		x = self.fc1(x)

        Massive change, new model below
        def __init__(self):
		super(F102Classifier, self).__init__()
		
		self.pool = nn.MaxPool2d(8, 2)
		self.conv1 = nn.Conv2d(3, 6, 3) #3 inputs 6 hiddens
		self.batchnorm1 = nn.BatchNorm2d(6) #Normalizes above 
		self.conv2 = nn.Conv2d(6, 30, 3) # 12 hiddens
		self.batchnorm2 = nn.BatchNorm2d(30)
		self.conv3 = nn.Conv2d(30, 30, 9) # 12 hiddens
		self.batchnorm3 = nn.BatchNorm2d(30)
		self.fc1 = nn.Linear(1080, 204)
		self.dropout = nn.Dropout(0.2)
		self.fc2 = nn.Linear(204, 102) #102 output neurons
	def forward(self, x):
		x = self.pool(F.relu(self.conv1(x)))
		x = self.batchnorm1(x)
		x = self.pool(F.relu(self.conv2(x)))
		x = self.batchnorm2(x)
		x = self.pool(F.relu(self.conv3(x)))
		x = self.batchnorm3(x)
		x = x.view(training_batch_size, -1)
		x = F.relu(self.fc1(x))
		x = self.dropout(x)
		x = self.fc2(x)
		return x
        14 epochs = 42.5%
        
        Experimented with different crop size, 128 seems best. Also now are saving the models of the best epochs so good ones are not lost

		implementing random names, L1 and L2 regularisation
		Sck epoch 13, accuracy = 45%

		implementing additional augmentation with random perspectives

		Recognising the importance of high channel outputs, lol
				self.pool = nn.MaxPool2d(2, 2)
		self.conv1 = nn.Conv2d(3, 32, 3, bias = False, groups = 1) #3 inputs 6 hiddens
		self.batchnorm1 = nn.BatchNorm2d(32) #Normalizes above 
		self.conv2 = nn.Conv2d(32, 64, 3, bias = False, groups = 1) # 12 hiddens
		self.batchnorm2 = nn.BatchNorm2d(64)
		self.conv3 = nn.Conv2d(64, 64, 3, bias = False, groups = 1) # 12 hiddens
		self.batchnorm3 = nn.BatchNorm2d(64)
		self.conv4 = nn.Conv2d(64, 64, 3, bias = False, groups = 1) # 12 hiddens
		self.batchnorm4 = nn.BatchNorm2d(64)
		self.conv5 = nn.Conv2d(64, 64, 3, bias = False, groups = 1) # 12 hiddens
		self.batchnorm5 = nn.BatchNorm2d(64)
		self.fc1 = nn.Linear(2304, 2304)
		self.dropout = nn.Dropout(0.2)
		self.fc2 = nn.Linear(2304, 102) #102 output neurons
	def forward(self, x):
		x = F.relu(self.conv1(x))
		x = self.batchnorm1(x)
		x = self.pool(F.relu(self.conv2(x)))
		x = self.batchnorm2(x)
		x = self.pool(F.relu(self.conv3(x)))
		x = self.batchnorm3(x)
		x = self.pool(F.relu(self.conv4(x)))
		x = self.batchnorm4(x)
		x = self.pool(F.relu(self.conv5(x)))
		x = self.batchnorm5(x)
		x = x.view(training_batch_size, -1)
		x = F.relu(self.fc1(x))
		#x = self.dropout(x)
		x = self.fc2(x)
		return x
		learning_rate = 0.001
		lambda1 = 0.001
		weightDecay = 0.001
		#momentum = 0.9
		crop_size = 128

		Tmk shown in discord has 47% (peak of 55% but took a while)
		Exx has 128 output channels in final layers but seems to overfit more, 44%
		Qba is Exx but with dropout, accuracy is increasing and overfitting less, 50%

 """

In [ ]:
Pmodel = F102Classifier()
Pmodel.load_state_dict(torch.load("./models/classifierTmkEpoch81Accuracy54.pth"))
test(validation_dataloader, Pmodel, loss_function)

this label tensor(28, device='cuda:0') missclassed astensor(8, device='cuda:0')
this label tensor(10, device='cuda:0') missclassed astensor(87, device='cuda:0')
this label tensor(89, device='cuda:0') missclassed astensor(45, device='cuda:0')
this label tensor(44, device='cuda:0') missclassed astensor(2, device='cuda:0')
this label tensor(47, device='cuda:0') missclassed astensor(14, device='cuda:0')
this label tensor(10, device='cuda:0') missclassed astensor(96, device='cuda:0')
this label tensor(37, device='cuda:0') missclassed astensor(78, device='cuda:0')
this label tensor(38, device='cuda:0') missclassed astensor(71, device='cuda:0')
this label tensor(78, device='cuda:0') missclassed astensor(9, device='cuda:0')
this label tensor(61, device='cuda:0') missclassed astensor(95, device='cuda:0')
this label tensor(34, device='cuda:0') missclassed astensor(54, device='cuda:0')
this label tensor(24, device='cuda:0') missclassed astensor(3, device='cuda:0')
this label tensor(37, device='cu

0.546078431372549